In [1]:
import numpy as np
import json

import torch
from torch import nn
from torch import optim
from torchvision import datasets, transforms, models
from torch.utils.data.sampler import SubsetRandomSampler

from collections import OrderedDict

In [2]:
data_dir = 'flower_data'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
##########################################################################
# Prepare datasets
##########################################################################
# TODO: Define your transforms for the training and validation sets
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.Resize(255),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

# TODO: Load the datasets with ImageFolder
train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
test_dataset = datasets.ImageFolder(valid_dir, transform=test_transforms)

'''Dataset split'''
batch_size = 64
valid_size = 0.1

# obtain training indices that will be used for validation
indices = list(range(len(train_dataset)))
np.random.shuffle(indices)
split = int(np.floor(valid_size * len(train_dataset)))
train_idx, valid_idx = indices[split:], indices[:split]

# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# TODO: Using the image datasets and the trainforms, define the dataloaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler)
valid_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, sampler=valid_sampler)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [3]:
'''Label mapping'''
with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

In [4]:
##########################################################################
# Create model
##########################################################################
model = models.resnet152(pretrained=True)

# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False
"""
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(2048, 512)),
                          ('relu', nn.ReLU()),
                          ('dropout1', nn.Dropout(0.3)), # To avoid over-fitting
                          ('fc2', nn.Linear(512, 102)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
model.fc = classifier
"""
model.fc = nn.Sequential(nn.Linear(2048, 512),
                         nn.ReLU(),
                         nn.Dropout(0.6),
                         nn.Linear(512, 256),
                         nn.ReLU(),
                         nn.Dropout(0.4),
                         nn.Linear(256, 102),
                         nn.Dropout(0.2),
                         nn.LogSoftmax(dim=1))
    
criterion = nn.NLLLoss()
# Only train the classifier parameters, feature parameters are frozen
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

In [5]:
'''Load the checkpoint'''
# TODO: Write a function that loads a checkpoint and rebuilds the model
model.load_state_dict(torch.load('model.pth'))
print("Our model: \n\n", model, '\n')
print("The state dict keys: \n\n", model.state_dict().keys())

Our model: 

 ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=

) 

The state dict keys: 



 odict_keys(['conv1.weight', 'bn1.weight', 'bn1.bias', 'bn1.running_mean', 'bn1.running_var', 'bn1.num_batches_tracked', 'layer1.0.conv1.weight', 'layer1.0.bn1.weight', 'layer1.0.bn1.bias', 'layer1.0.bn1.running_mean', 'layer1.0.bn1.running_var', 'layer1.0.bn1.num_batches_tracked', 'layer1.0.conv2.weight', 'layer1.0.bn2.weight', 'layer1.0.bn2.bias', 'layer1.0.bn2.running_mean', 'layer1.0.bn2.running_var', 'layer1.0.bn2.num_batches_tracked', 'layer1.0.conv3.weight', 'layer1.0.bn3.weight', 'layer1.0.bn3.bias', 'layer1.0.bn3.running_mean', 'layer1.0.bn3.running_var', 'layer1.0.bn3.num_batches_tracked', 'layer1.0.downsample.0.weight', 'layer1.0.downsample.1.weight', 'layer1.0.downsample.1.bias', 'layer1.0.downsample.1.running_mean', 'layer1.0.downsample.1.running_var', 'layer1.0.downsample.1.num_batches_tracked', 'layer1.1.conv1.weight', 'layer1.1.bn1.weight', 'layer1.1.bn1.bias', 'layer1.1.bn1.running_mean', 'layer1.1.bn1.running_var', 'layer1.1.bn1.num_batches_tracked', 'layer1.1.conv2.w

In [6]:
'''Choose device'''
device = torch.device("cuda" if torch.cuda.is_available() else "cpu"); print(device)

cuda


In [8]:
##########################################################################
# Test model
##########################################################################
model.to(device)
model.eval()

test_epoches = 2

class_num = [0]*102
class_accuracy = [0]*102

for e in range(test_epoches):
    with torch.no_grad():
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)
            
            test_accuracy = 0
            
            # log_ps.shape should be (64, 102) batch size * class size
            log_ps = model.forward(images)
            loss = criterion(log_ps, labels)
            ps = torch.exp(log_ps) 
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
            
            for idx, lbl in enumerate(labels):
                class_num[lbl.item()] += 1
                class_accuracy[lbl.item()] += equals[idx].item()
                
print('{:<35}{:>5}/{:<10} {:>10.2f}%'.format('Total accuracy:', np.sum(class_accuracy), np.sum(class_num), 
                                 np.sum(class_accuracy)/np.sum(class_num)*100))
print('Accuracy per class: ')
for idx, num in enumerate(class_num):
    cls = train_dataset.classes[idx]
    nam = cat_to_name[cls]
    acc = class_accuracy[idx]
    acc_rate = float(acc) / num if num != 0 else 0
    print('{:<3} {:<30} {:>5}/{:<10} {:>10.2f}%'.format(cls, nam, acc, num, acc_rate*100))

Total accuracy:                      950/1310            72.52%
Accuracy per class: 
1   pink primrose                      0/0                0.00%
10  globe thistle                      6/8               75.00%
100 blanket flower                     2/6               33.33%
101 trumpet creeper                    7/10              70.00%
102 blackberry lily                   15/16              93.75%
11  snapdragon                         5/12              41.67%
12  colt's foot                        8/8              100.00%
13  king protea                        4/4              100.00%
14  spear thistle                     10/10             100.00%
15  yellow iris                        7/10              70.00%
16  globe-flower                       5/10              50.00%
17  purple coneflower                  6/6              100.00%
18  peruvian lily                      7/14              50.00%
19  balloon flower                     2/6               33.33%
2   hard-leaved poc

In [11]:
##########################################################################
# Information block
##########################################################################
lbl = labels[0].item()
print(type(lbl), 'class: {},'.format(train_dataset.classes[lbl]), 
                 'name: {}'.format(cat_to_name[train_dataset.classes[lbl]]), '\n')
print('Actual label index: ', [lbl.item() for lbl in labels])
print('Predicted label index: ', [cls.item() for cls in top_class])
print('Right predictions: ', [eql.item() for eql in equals], '\n')
print('Total right prediction number: ', torch.sum(equals).item())
print('Total predictions: ', len(labels))

<class 'int'> class: 8, name: bird of paradise 

Actual label index:  [80, 70, 49, 87, 82, 73, 16, 25, 83, 96, 6, 56, 4, 89, 52]
Predicted label index:  [80, 70, 49, 87, 82, 73, 60, 67, 13, 96, 6, 56, 4, 78, 52]
Right predictions:  [1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1] 

Total right prediction number:  11
Total predictions:  15


In [10]:
##########################################################################
# Information block
##########################################################################
class_num = [0]*102
class_accuracy = [0]*102

for idx, lbl in enumerate(labels):
    class_num[lbl.item()] += 1
    class_accuracy[lbl.item()] += equals[idx].item()

print('Item number per class: ', class_num, '\n')
print('Right prediciton number per class: ', class_accuracy, '\n')
print('Verify class number: ', np.sum(class_num))
print('Verify right prediction number: ', np.sum(class_accuracy))
print('Accuracy of this batch: ', np.sum(class_accuracy)/np.sum(class_num))

Item number per class:  [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0] 

Right prediciton number per class:  [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0] 

Verify class number:  15
Verify right prediction number:  11
Accuracy of this batch:  0.7333333333333333
